In [ ]:
import json
import datetime
import altair as alt
import pandas as pd
import numpy as np
import os

hits = [
# '2018-08-10_00-00-00', #0
# '2018-08-29_11_32_44',
'2018-08-29_22-54-16', #2
'2018-09-05_21-38-43',
'2018-09-07_11-34-48',
'2018-09-08_09-41-18',
'2018-09-09_10-28-14', #6
'2018-09-10_10-36-44', #7
'2018-09-11_11-09-18',
#'2018-09-12_11-04-34',
'2018-09-13_12-06-31',
]
# base = 'hits/' + hits[8]
def cd(file):
    return os.path.join(base, file)

In [ ]:
def duration_plot(assignments):
    times = []
    submit_times = []
    for l in assignments:
        submit = datetime.datetime.strptime(l['SubmitTime'][:-6], "%Y-%m-%d %H:%M:%S")
        accept = datetime.datetime.strptime(l['AcceptTime'][:-6], "%Y-%m-%d %H:%M:%S")
        submit_times.append(submit)
        diff = (submit - accept).total_seconds()/60.0
        times.append({"diff": diff})
    spec = {
      "data": {"values": times},
      "mark": "bar",
      "encoding": {
        "x": {
          "field": "diff",
          "type": "quantitative",
          "bin": {"step": 1} 
        },
        "y": {
          "aggregate": "count",
          "type": "quantitative"
        }
      }
    }
    print('min - max: ', min(submit_times), max(submit_times))
    print('mean duration: ', np.mean(list(map(lambda d: d['diff'], times))))
    return alt.VegaLite(spec)

In [ ]:
listener_assignments = json.load(open(cd('listener.assignments')))
speaker_assignments = json.load(open(cd('speaker.assignments')))
display(duration_plot(speaker_assignments))
display(duration_plot(listener_assignments))

In [ ]:
import collections 

def by_worker(assign):
    workers = collections.defaultdict(int)
    for a in assign:
        workers[a['WorkerId']] += 1
    descending = sorted(workers.items(), key=lambda a: -a[1])
    return collections.OrderedDict(descending) 

listener_counts = by_worker(listener_assignments)
speaker_counts = by_worker(speaker_assignments)
intersection = set(speaker_counts.keys()).intersection(listener_counts.keys())

for k in intersection:
    print(k, listener_counts[k], speaker_counts[k])
print('speak: {} listener {} overlap {}'.format(len(speaker_counts), len(listener_counts), len(intersection)))

In [ ]:
import pandas as pd
import collections
from scripts.query_line import QueryLine
pd.set_option('display.max_colwidth', 200)
pd.set_option("display.max_rows", None)

def isfloat(string):
    try:
        float(string)
        return True
    except ValueError:
        return False
    
def reduce_formula(x):
    if x.find(':') == -1:
        print('COLON_NOT_FOUND', x)
        return 'COLON_NOT_FOUND'
    v = x[x.index(':')+1:]
    return x[:x.index(':')] + ('number' if isfloat(v) else v)

def to_html(spec):
    options = {'rendered': 'svg', 'width': 800, 'height': 800}
    return alt.utils.html.spec_to_html(spec, 'vega-lite', '3', '3', '2', embed_options=options)


In [ ]:
from scripts import process_listener as pl
from scripts import guess_id as guess
# adds annotations

def aggregate_all():
    examples = []
    listener_log = []
    for h in hits[0:1] + hits[2:]:
        print('processing', h)
        speaker_path = os.path.join('hits', h, 'speaker.jsonl')
        listener_path = os.path.join('hits', h, 'listener.raw.jsonl')
        lines = [QueryLine(line.strip()) for line in open(speaker_path, 'r').readlines()]
        examples += [l for l in lines if l.is_example()]
        listener_log += [QueryLine(line.strip()) for line in open(listener_path, 'r').readlines()]
#     formulas = [l.example()['targetFormula'] for l in examples]
#     paths = list(map(lambda x: reduce_formula(x), formulas))
    paths = [reduce_formula(l.example()['targetFormula'])
             + guess.hash_query(l.json, keys=['context']) for l in examples]
    pathsset = set(paths)
    pathcounter = collections.Counter(paths)
    countisone = [k for k in pathcounter.keys() if pathcounter[k] == 1]
    freqfreq = pathcounter.values()
    spec = {
      "data": {"values": [{"freq_of_freq": v} for v in freqfreq]},
      "mark": "bar",
      "encoding": {
        "x": {
          "field": "freq_of_freq",
          "type": "quantitative",
          "bin": {"step": 1} 
        },
        "y": {
          "aggregate": "count",
          "type": "quantitative"
        }
      }
    }
    chart = alt.VegaLite(spec)
    display(chart)
    print(len(paths), len(pathsset), len(countisone))
    
    pl.process_listener(examples, listener_log)
    speakers, listeners = pl.aggregate_turker(examples)
    print('got {} examples, {} listeners log'.format(len(examples), len(listener_log)))
    return speakers, listeners

speakers, listeners = aggregate_all()

In [ ]:
def hist(data, field):
    records = [{field: i} for i in data]
    spec = {
      "data": {"values": records},
      "mark": "bar",
      "encoding": {
        "x": {
          "field": field,
          "type": "quantitative",
          "bin": {"step": 1} 
        },
        "y": {
          "aggregate": "count",
          "type": "quantitative"
        }
      }
    }
    return alt.VegaLite(spec)

def type2acc(stats):
    total = stats['correct'] + stats['wrong'] + stats['skip'] 
    if total != 0:
        correct = (stats['correct'])/(total)
        wrong = (stats['wrong']) / (total)
        skip = (stats['skip']) /(total)
        return correct, wrong, skip
    else:
        return 0,0,0
    
    
def print_stats():
    overlaps = set(speakers.keys()).intersection(listeners.keys())
    print(f's_count, l_count {len(speakers)} {len(listeners)}')
    print(f'{len(overlaps)} in common')
    speaker_utts = [len(s['utterances']) for k,s in speakers.items()]
    display(hist(speaker_utts, 'utts per speaker'))
    print(f'mean, median number of utterances {np.mean(speaker_utts):.3f} {np.median(speaker_utts)}')
    all_utts = [u for k,s in speakers.items() for u in s['utterances']]
    print('total {} utterances, mean_len {:.2f} chars, {:.2f} tokens'.format(
        len(all_utts),
        np.mean([len(u) for u in all_utts]),
        np.mean([len(u.split(' ')) for u in all_utts])))
    all_listeners = [u for k,s in speakers.items() for u in s['listeners']]
    print(len(all_listeners))
    types = pl.aggregate_type(all_listeners)
    print('correct, wrong, skip: {:.3f}, {:.3f}, {:.3f}'.format(*type2acc(types)))
    
print_stats()

In [ ]:
def q2row(l: QueryLine):
    return {'utterance': l.utterance(), 'workerId': l.worker_id(), 'targetFormula': l.example()['targetFormula']} 

def speaker2row(id, l: dict):
    return {'WorkerId': id, 'stats': l['stats'],
            'utts': ', '.join(l['utterances']), 'listeners': len(l['listeners'])}

def joint2row(id, l: dict):
    l['WorkerId'] = id
    return l

joint = collections.defaultdict(lambda: {'WorkerId':'', 'total': 0})

def fill_joint(rows, prefix):
    for wid, r in rows.items():
#         print('row', r)
        jr = joint[wid]
        jr['WorkerId'] = wid
        stats = r['stats']
        total = stats['correct'] + stats['wrong'] + stats['skip'] 
        correct, wrong, skip = 0.2, 0.6, 0.2
        if total != 0:
            correct = (stats['correct']) /  (total)
            wrong = (stats['wrong']) /  (total)
            skip = (stats['skip']) /  (total)
        jr[prefix + '_correct'] = correct 
        jr[prefix + '_wrong'] = wrong
        jr[prefix + '_skip'] = skip
        jr[prefix + '_total'] = total
        jr['total'] += total
        
def fill_utts(speakers):
    for wid, r in speakers.items():
#         print('row', r)
        jr = joint[wid]
        jr['WorkerId'] = wid
        utts = r['utterances']
        lenutt = len(utts)
        jr['num_utts'] = lenutt 
        jr['utts'] = utts if lenutt <= 10 else utts[0:2]
        jr['avg_len'] = np.mean([len(l.split(' ')) for l in utts])
        
fill_joint(speakers, 's')
fill_utts(speakers)
fill_joint(listeners, 'l')


records = [joint2row(k, v) for k,v in joint.items()]
agg_table = pd.DataFrame.from_records(records) 
agg_table.sort_values(by=['s_correct', 's_total', 'l_correct'], inplace=True, ascending=False)
# display(agg_table)
spec_list = []
spec_list += [{
  "data": {"values": records},
  "description": "accuracies",
  "mark": "circle",
  "encoding": {
    "x": {
      "field": "s_correct",
      "type": "quantitative"        
    },
    "y": {
      "field": "l_correct",
      "type": "quantitative"        
    },
    "size": {
        "field": "num_utts",
        "type": "quantitative"
    },
    "opacity": {"value": 0.5},
    "tooltip": {
        "field": "l_correct",
        "type": "quantitative"
    }
  }
}]
spec_list += [{
  "data": {"values": records},
  "description": "correlation",
  "mark": "circle",
  "encoding": {
    "x": {
      "field": "s_correct",
      "type": "quantitative"        
    },
    "y": {
      "field": "l_total",
      "type": "quantitative"        
    },
    "size": {
        "field": "num_utts",
        "type": "quantitative"
    },
    "opacity": {"value": 0.5},
    "tooltip": {
        "field": "utts",
        "type": "nominal"
    }
  }
}]

chart = alt.VegaLite(spec_list[0])
display(chart)

for spec in spec_list:
    html = to_html(spec)
    name = spec['description']
    open(name + '.html', 'w').write(to_html(spec))

corr = agg_table.corr()
display(corr)
display(agg_table)

In [ ]:
ex_table = pd.DataFrame.from_records([speaker2row(k, v) for k,v in speakers.items()])
ex_table.sort_values(by=['listeners'])
display(ex_table)
open('visualize.html', 'w').write(table.to_html())